In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, minimize_scalar
import emcee
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
import seaborn as sns
import sys
sys.path.append("..")
from src.utils import contact_matrix
from src.dataset import CRKPTransmissionSimulator

from timeit import default_timer as timer

Computing the exact likelihood for the CRKP transmission model.

$$
    \log \mathcal{L}(\beta ; \mathbf{X}) = \sum_{t=2}^T \sum_{i=1}^N [(1 - X_{t-1}^{(i)})W_t^{(i)}W_{t-1}^{(i)}] \cdot [X_t^{(i)} \log (1 - e^{- \lambda_i(t)}) + (1 - X_t^{(i)}) \log e^{- \lambda_i(t)})]
$$

In [ ]:
path = "/Volumes/umms-esnitkin/Project_KPC_LTACH/Analysis/LTACH_transmission_modeling/preprocessed/resampled"

In [ ]:
load_homog = True
load_het = True

In [ ]:
X = pd.read_csv(f"{path}/infections.csv", index_col=0).values
W = pd.read_csv(f"{path}/facility_trace.csv", index_col=0).values
V = pd.read_csv(f"{path}/screening.csv", index_col=0).values
F = pd.read_csv(f"{path}/floor_trace.csv", index_col=0).values
R = pd.read_csv(f"{path}/room_trace.csv", index_col=0).values
cap = np.array([129., 28., 38., 35., 27., 17., 2])

In [ ]:
with open(f"{path}/observed_data.npy", "rb") as f:
            x_o = (np.load(f).T / cap).T

In [ ]:
def log_likelihood(logbeta, X, W, V, F, R, het=False):
    beta = np.exp(logbeta)
    beta_0 = beta[0] if het else beta
    N, T = W.shape
    # X = np.empty((N, T))
    # load screen data for first day
    ans = 0
    for t in range(1, T):
        x = X[:, t-1]
        w = W[:, t-1]
        f = F[:, t-1]
        r = R[:, t-1]
        # who was infected at the last timestep?
        Ia = x[w > 0]
        hazard = Ia.sum() * beta_0 * np.ones(N) / cap[0]
        fa = f[w > 0]
        fC = contact_matrix(fa)
        # guarantee that there are no infecteds who aren't present
        # how many infected floormates?
        if het:
            hazard[w > 0] += (fC * Ia).sum(1) * beta[fa] / cap[fa]
        ra = r[w > 0]
        rC = contact_matrix(ra)
        infected_roommates = (rC * Ia).sum(1)
        if het:
            hazard[w > 0] += infected_roommates * beta[-1] / cap[-1]
        p = 1 - np.exp(-hazard)
        
        xt = X[:, t]
    
        valid = W[:, t] * w * (1 - x) # present for at least week & susceptible

        ans += np.nansum(valid * ((xt * np.log(p)) + (1 - xt) * np.log(1 - p)))


    return ans

# Maximum Likelihood Estimation

### homogeneous model

In [ ]:
def f(logbeta):
    return - log_likelihood(logbeta, X, W, V, F, R)

In [ ]:
res = minimize_scalar(f)
np.exp(res.x)

### heterogeneous model

In [ ]:
def g(logbeta):
    return - log_likelihood(logbeta, X, W, V, F, R, True)

In [ ]:
res2 = minimize(g, x0 = np.full(7, -3), tol=0.0001)
np.round(np.exp(res2.x), 3)

# MCMC

### Homogeneous Model

In [ ]:
def log_prob_homog(logbeta, X, W, V, F, R, prior_mu):
    prior = norm(prior_mu)
    lp = prior.logpdf(logbeta)
    # lp = log_prior(np.log(beta))
    return lp + log_likelihood(logbeta, X, W, V, F, R)

In [ ]:
if load_homog:
    flat_samples = np.load("mcmc_homog.npy")
else:
    nwalkers = 4
    pos = res.x + 1e-4 * np.random.randn(nwalkers, 1)
    prior_mu = -2
    sampler = emcee.EnsembleSampler(
        nwalkers, 1, log_prob_homog, args = (X, W, V, F, R, prior_mu)
    )
    sampler.run_mcmc(pos, 2000, progress=True)

    flat_samples = sampler.get_chain(discard=100, thin=50, flat=True)
    np.save("mcmc_homog", flat_samples)
    

MCMC estimate uses 8,000 evaluations of the likelihood function.

In [ ]:
print(flat_samples.mean())
print(flat_samples.std())
print(np.exp(flat_samples).mean())

In [ ]:
np.exp(-2.09)

In [ ]:
np.median(np.exp(flat_samples))

In [ ]:
plt.hist(flat_samples)
plt.show()

In [ ]:
# TODO: figure out how to benchmark homogeneous posterior predictive check

K = 30
T = 53
crkp_model = CRKPTransmissionSimulator(path, -3, 1, heterogeneous=False)
# neural_posterior_het = multivariate_normal(mu, sigma)
# npe_sample_het = neural_posterior_het.rvs(size=K, random_state=2)
posterior_predictive_mcmc = np.empty((K, 7, T))
for i in range(K):
    x_rep = crkp_model.CRKP_simulator(flat_samples[i], i, summarize=True, show_full=True) * np.repeat(cap[:, None], T, 1) 
    posterior_predictive_mcmc[i] =  np.array(x_rep)

In [ ]:
# calculate mean squared error
for j in range(7):
    preds = posterior_predictive_mcmc[:, j, :]
    target = (x_o[j] * cap[j])
    mse = ((preds - target)**2).mean()
    print(round(mse, 3))

In [ ]:
for j in range(7):
    pp_mean = posterior_predictive_mcmc.mean(0)[j]
    labels = ["Post. Pred. Draw"] + [None for _ in range(K-1)]
    I_o = np.array(x_o)[j] * cap[j]
    plt.plot(I_o, label="Observed", color="k")
    plt.plot(posterior_predictive_mcmc[:, j, :].T, 
             label=labels, color="b",
             alpha=0.1)
    plt.plot(pp_mean, label="Post. Pred. Mean", linestyle="--",
             color="orange")
    # sns.lineplot(NN, color="green", linestyle="--", label="Total")
    plt.legend()
    if j == 6:
        ylab = "Infected Rooms"
    else:
        ylab = "Infected Patients"
    plt.ylabel(ylab)
    plt.xlabel("Weeks")
    # plt.savefig(f"images/crkp/crkp_ppc_abc{j}.png")
    plt.show()

### Heterogeneous Model

In [ ]:
def log_prob_het(logbeta, X, W, V, F, R, prior_mu):
    prior = multivariate_normal(np.full(7, prior_mu))
    lp = prior.logpdf(logbeta)
    return lp + log_likelihood(np.exp(logbeta), X, W, V, F, R, het=True)

In [ ]:
np.clip(res2.x, a_min=-5, a_max=None)

In [ ]:
if load_het:
    flat_samples_het = np.load("mcmc_het.npy")
else:
    nwalkers = 16
    initial = np.clip(res2.x, a_min=-5, a_max=None) # np.full(7, -3)
    pos = initial + 1e-1 * np.random.randn(nwalkers, 7)
    prior_mu = -3
    sampler_het = emcee.EnsembleSampler(
        nwalkers, 7, log_prob_het, args = (X, W, V, F, R, prior_mu)
    )
    # up to 4,000
    sampler_het.run_mcmc(pos, 1000, progress=True, skip_initial_state_check=True)
    flat_samples_het = sampler_het.get_chain(discard=500, thin=100, flat=True)
    np.save("mcmc_het", flat_samples_het)

In [ ]:
np.exp(flat_samples_het).mean(0)

In [ ]:
(flat_samples_het).mean(0)

In [ ]:
# convert to relative risks
w = mcmc_pe / cap 
w / w[0]

## Posterior Predictive Checks

In [ ]:
K = 30
T = 53
crkp_model = CRKPTransmissionSimulator(path, -3, 1, heterogeneous=True)
# neural_posterior_het = multivariate_normal(mu, sigma)
# npe_sample_het = neural_posterior_het.rvs(size=K, random_state=2)
sample_ix = np.random.choice(flat_samples_het.shape[0], 30, False)
posterior_predictive_mcmc_het = np.empty((K, 7, T))
for i in range(K):
    x_rep = crkp_model.CRKP_simulator(flat_samples_het[sample_ix[i]], i, summarize=True) * np.repeat(cap[:, None], T, 1) 
    posterior_predictive_mcmc_het[i] =  np.array(x_rep)

In [ ]:
# calculate mean squared error
for j in range(7):
    preds = posterior_predictive_mcmc_het[:, j, :]
    target = (x_o[j] * cap[j])
    mse = ((preds - target)**2).mean()
    print(round(mse, 3))

In [ ]:
for j in range(7):
    pp_mean = posterior_predictive_mcmc_het.mean(0)[j]
    labels = ["Post. Pred. Draw"] + [None for _ in range(K-1)]
    I_o = np.array(x_o)[j] * cap[j]
    plt.plot(I_o, label="Observed", color="k")
    plt.plot(posterior_predictive_mcmc_het[:, j, :].T, 
             label=labels, color="b",
             alpha=0.1)
    plt.plot(pp_mean, label="Post. Pred. Mean", linestyle="--",
             color="orange")
    # sns.lineplot(NN, color="green", linestyle="--", label="Total")
    plt.legend()
    if j == 6:
        ylab = "Infected Rooms"
    else:
        ylab = "Infected Patients"
    plt.ylabel(ylab)
    plt.xlabel("Weeks")
    # plt.savefig(f"images/crkp/crkp_ppc_abc{j}.png")
    plt.show()

# NPE

In [ ]:
# model.d_model=32,48,64,model.embed_dim=8,16,model.weight_decay=0,0.01,0.02

# best run: d_model = 48, d_embed = 8, weight_decay = 0

# how many samples?
mu = -2.069
sigma = 0.163

# ABC Sanity Check

In [ ]:
def abc_rejection_sampler(S, epsilon, prior_sampler, simulator, 
                          x_o, max_attempts=10000, summarize=False,
                          print_every=5000, error_scaling = None):
    # S: total number of particles
    samples = []
    attempts = 0
    # x_o is shape (d_theta, d_x)
    errors = np.full(max_attempts, -1e3)
    start_time = timer()
    for s in range(S):
        accept = False
        while not accept:
            theta = np.array(prior_sampler())[0]
            x = simulator(theta, seed=attempts)
            accept, error = accept_sample(x, x_o, epsilon, summarize, error_scaling)
            if accept:
                samples.append(theta)
                # if s % 10 == 0: print(s)
            errors[attempts] = error
            attempts += 1
            if attempts == max_attempts:
                print("Maximum attempts reached, halting")
                return np.array(samples), errors
            if not attempts % print_every:
                print(f"Attempts: {attempts:,}")
    end_time = timer()
    accept_rate = S / attempts
    
    print(f"Time lapsed: {end_time - start_time:.2f} seconds")
    print(f"With tolerance {epsilon}, acceptance rate: {accept_rate:.6f}")
    print(f"Total number of attempts: {attempts:,}")
    return np.array(samples), errors

def accept_sample(x, x_o, epsilon, summarize, lam):
    if summarize:
        x_o = x_o[0]
    v = np.array(x - x_o)
    if lam is not None:
        v = v * lam.reshape(-1, 1)
    error = np.linalg.norm(v)
    accept = (error < epsilon)

    return accept, error

In [ ]:
summarize = False #True
hetero = True

prior_mu = -2
prior_sigma = 1
model = CRKPTransmissionSimulator(path, prior_mu, prior_sigma,
                                  heterogeneous=False)


prior_sampler = lambda: model.sample_logbeta(1)
simulator = lambda theta, seed:(model.CRKP_simulator(theta, seed))[W > 0]
# simulator = lambda theta, seed: np.nansum(model.CRKP_simulator(theta, seed), 0)

In [ ]:
obs = X[W > 0] # np.nansum(X, 0)
abc_samples, error = abc_rejection_sampler(100, 20, prior_sampler, simulator, 
                          obs, max_attempts=1000000, summarize=False,
                          print_every=1000, error_scaling = None)

# Data Exploration

Checking: novel infections on each floor.


In [ ]:
N, T = F.shape
def count_acquired(f):
    imported = 0
    acquired = 0
    for i in range(N):
        present = False
        for t in range(T):
            if F[i, t] == f:
                if X[i, t] == 1 and not present:
                    imported += 1
                    break
                elif X[i, t] == 1 and present:
                    acquired += 1
                    break
                present = True
            else:
                present = False
    return imported, acquired

In [ ]:
imported = 0
acquired = 0
floor_ac = np.array([0,0,0,0,0])
floor_im = np.array([0,0,0,0,0])
ac_ix = []
for i in range(N):
    present = False
    for t in range(T):
        if W[i, t] == 1:
            if X[i, t] == 1 and not present:
                imported += 1
                f = F[i, t]
                floor_im[f-1] += 1
                break
            elif X[i, t] == 1 and present:
                acquired += 1
                f = F[i, t]
                floor_ac[f-1] += 1
                ac_ix.append(i)
                break
            present = True
        else:
            present = False
imported, acquired

In [ ]:
# cool summary stats
Y = np.empty(X.shape)
Y[:] = np.nan
for i in range(N):
    present = False
    novel_infection = False
    for t in range(T):
        if W[i, t] == 1:
            if X[i, t] == 1:
                # susceptible = False
                if present:
                    novel_infection = True
                if novel_infection:
                    Y[i, t] = 1
                else:
                    Y[i, t] = 0
            else:
                present = True
                Y[i, t] = 0
        else:
            present = False

In [ ]:
set2 = set(np.array(range(N))[(Y == 1).any(1)])

In [ ]:
set2 - set1

In [ ]:
X[209]

In [ ]:
X[248]

In [ ]:
f = 2
plt.plot(np.sum(W * (F == f), 0))
plt.plot(np.sum(np.nan_to_num(X, 0) * (F == f), 0))
plt.plot(np.sum(np.nan_to_num(Y, 0) * (F == f), 0))

In [ ]:
# what floors are experiencing outflow?

In [ ]:
flow_matrix = np.zeros((5, 5))
starting_floor = np.zeros(5)
samples = {1:[], 2:[], 3:[], 4:[], 5:[]}
for i in range(N):
    for t in range(1,T):
        old_floor = F[i, t-1]
        new_floor = F[i, t]
        if old_floor and new_floor:
            if old_floor != new_floor:
                flow_matrix[old_floor-1, new_floor-1] += 1
        else:
            if new_floor and not old_floor:
                starting_floor[new_floor - 1] += 1
                samples[new_floor].append(i)

In [ ]:
for i in range(1, 6):
    plt.plot((W * (F == i)).sum(0), label=f"{i}")

plt.legend()